In [1]:
import spacy
import pytextrank

/home/khal-drog0/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-08-21 01:03:21.731479: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-21 01:03:21.731654: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-08-21 01:03:25.050863: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-08-21 01:03:25.051157: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcu

In [2]:
# ! python3 -m spacy download en_core_web_lg

nlp = spacy.load('en_core_web_lg')

In [3]:
nlp.add_pipe('textrank')

In [4]:
example_text = 'Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks and convolutional neural networks have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance. Artificial neural networks (ANNs) were inspired by information processing and distributed communication nodes in biological systems. ANNs have various differences from biological brains. Specifically, neural networks tend to be static and symbolic, while the biological brain of most living organisms is dynamic (plastic) and analogue. The adjective "deep" in deep learning refers to the use of multiple layers in the network. Early work showed that a linear perceptron cannot be a universal classifier, but that a network with a nonpolynomial activation function with one hidden layer of unbounded width can. Deep learning is a modern variation which is concerned with an unbounded number of layers of bounded size, which permits practical application and optimized implementation, while retaining theoretical universality under mild conditions. In deep learning the layers are also permitted to be heterogeneous and to deviate widely from biologically informed connectionist models, for the sake of efficiency, trainability and understandability, whence the structured part.'

In [5]:
example_text

'Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks and convolutional neural networks have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance. Artificial neural networks (ANNs) were inspired by information processing and distributed communication nodes in biological systems. ANNs have various differences from biological brains. Specifically, neural networks tend to be static and symbolic, while the biologic

In [6]:
doc = nlp(example_text)

In [7]:
for sentence in doc._.textrank.summary(limit_sentences = 2):
    print(sentence)

Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks and convolutional neural networks have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.
Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning.


In [8]:
phrases_and_ranks = [(phrase.chunks[0], phrase.rank) for phrase in doc._.phrases]

phrases_and_ranks[:10]

[(deep neural networks, 0.1043624955663785),
 (neural networks, 0.10058782649290761),
 (Artificial neural networks, 0.09934056204763708),
 (artificial neural networks, 0.09934056204763708),
 (machine learning methods, 0.09726306044569297),
 (convolutional neural networks, 0.09726243749515844),
 (recurrent neural networks, 0.09667514118373426),
 (deep structured learning, 0.09662536048952462),
 (human expert performance, 0.09210992513092017),
 (deep belief networks, 0.09198485548110959)]

We are going to use 'Pegasus' which is a transformer model
It will predict and generate a new sentence from the input sentence
and the sentiment obtained from the sentence


In [9]:
# ! pip3 install --user transformers

In [10]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [11]:
model_name = 'google/pegasus-xsum'

In [12]:
# ! pip3 install --user sentencepiece

In [13]:
pegasus_tokenizer = PegasusTokenizer.from_pretrained(model_name)

In [14]:
pegasus_model = PegasusForConditionalGeneration.from_pretrained(model_name)

KeyboardInterrupt: 

In [ ]:
tokens = pegasus_tokenizer(example_text, truncation = True, 
                           padding = 'longest', return_tensors = 'pt')
tokens

In [ ]:
encoded_summary = pegasus_model.generate(**tokens)

In [ ]:
decoded_summary = pegasus_tokenizer.decode(encoded_summary[0], skip_special_tokens = True)

In [ ]:
decoded_summary

In [ ]:
from transformers import pipeline

summarizer = pipeline('summarization', model = model_name,
                      tokenizer = pegasus_tokenizer, 
                      framwork = 'pt')

In [ ]:
summary = summarizer(example_text, min_length = 30, max_length = 150)

In [ ]:
summary[0]['summary_text']